<h1><center>Laboratorio 7: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Rodrigo Mella S.
- Nombre de alumno 2: Miguel Morales M.


### **Link de repositorio de GitHub:** https://github.com/migmoralesmar/MDS7202/tree/Laboratorio7

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- Fecha de entrega: TBD
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
#!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# UMAP
# !pip install umap-learn

https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [2]:
df_players = pd.read_csv('stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [3]:
df_players['Labels'] = (~df_players['National_Position'].isna()).astype(int)
del df_players['National_Position']
df_players

,Name,Nationality,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,Skill_Moves,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Labels
0,Cristiano Ronaldo,Portugal,LW,185,80,Right,32,High / Low,4,5,...,95,85,92,93,90,81,76,85,88,1
1,Lionel Messi,Argentina,RW,170,72,Left,29,Medium / Medium,4,4,...,68,71,85,95,88,89,90,74,85,1
2,Neymar,Brazil,LW,174,68,Right,25,High / Medium,5,5,...,61,62,78,89,77,79,84,81,83,1
3,Luis Suárez,Uruguay,ST,182,85,Right,30,High / Medium,4,4,...,69,77,87,94,86,86,84,85,88,1
4,Manuel Neuer,Germany,GK,193,92,Right,31,Medium / Medium,4,1,...,78,25,25,13,16,14,11,47,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,Sub,183,82,Right,19,Medium / Medium,1,1,...,56,14,16,13,13,11,13,15,12,0
17584,Dylan McGoey,Republic of Ireland,Sub,185,80,Right,19,Medium / Medium,2,1,...,53,12,17,12,11,12,13,16,12,0
17585,Tommy Ouldridge,England,Res,173,61,Right,18,High / Medium,2,2,...,61,41,44,28,42,35,36,42,37,0
17586,Mark Foden,Scotland,Sub,180,80,Right,21,Medium / Medium,3,1,...,48,15,23,14,12,13,12,24,12,0


In [4]:
selected = (df_players['Labels']==1).sum()
noselected = (df_players['Labels']==0).sum()

print(f'Cantidad de jugadores seleccionados: {selected}.')
print(f'Cantidad de jugadores no seleccionados: {noselected}.')

Cantidad de jugadores seleccionados: 1075.
Cantidad de jugadores no seleccionados: 16513.


Como se puede observar hay un gran desbalance de los datos, teniendo 16513 ejemplos de jugadores no seleccionados y 1075 seleccionados.

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

In [6]:
coltransf = ColumnTransformer([
    ("MinMaxScaler",MinMaxScaler(),['Age','Shot_Power','Finishing','Long_Shots','Freekick_Accuracy','Penalties']),
    ("StandardScaler",StandardScaler(),['Height','Weight','Ball_Control','Dribbling','Aggression','Reactions','Vision',
                                        'Composure','Crossing','Short_Pass','Long_Pass','Acceleration','Speed','Stamina',
                                        'Balance','Agility','Jumping','Heading','Volleys']),
    ("OneHot",OneHotEncoder(handle_unknown='ignore'),['Nationality','Club_Position','Preffered_Foot',
                                                      'Work_Rate','Weak_foot','Skill_Moves']),
    ("PowerTransform",PowerTransformer(),['Marking','Sliding_Tackle','Standing_Tackle','Interceptions','Strength','Curve'])
])


- **MinMaxScaler:** aplicado a variables con tendencias marcadas (ascendente, descendente o constante) para normalizar sus valores en un rango de 0 a 1.
- **StandardScaler:** aplicado a variables con distribuciones similares a una gaussiana.
- **OneHotEncoder:** se aplica a variables categóricas.
- **PowerTransformer:** aplicado a variables con múltiples distribuciones gaussianas.

In [43]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

pipe1 = Pipeline([("Preprocess",coltransf),
      ('clf', KNeighborsClassifier())])

pipe2 = Pipeline([("Preprocess",coltransf),
      ('clf', SVC(kernel='linear',probability=True,random_state=1))])

pipe3 = Pipeline([("Preprocess",coltransf),
      ('clf', DecisionTreeClassifier(criterion='gini',random_state=1))])


### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [44]:
X = df_players.iloc[:,1:-1]
y = df_players['Labels']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y, random_state=1)

target_names = ['No clasificados', 'Clasificados']

print('Pipeline 1: KNeighborsClassifier')
pipe1.fit(X_train, y_train)
y_pred1 = pipe1.predict(X_test)
print(classification_report(y_test, y_pred1, target_names=target_names),'\n')
f1 = round(f1_score(y_test, y_pred1), 3)
print("F1 Score:", f1, end='\t')
kappa = round(cohen_kappa_score(y_test, y_pred1), 3)
print("Kappa:", kappa, end='\t')
accuracy = round(accuracy_score(y_test, y_pred1), 3)
print("Accuracy:", accuracy, '\n \n')


print('Pipeline 2: SVC linear')
pipe2.fit(X_train, y_train)
y_pred2 = pipe2.predict(X_test)
print(classification_report(y_test, y_pred2, target_names=target_names),'\n \n')
f1 = round(f1_score(y_test, y_pred2), 3)
print("F1 Score:", f1, end='\t')
kappa = round(cohen_kappa_score(y_test, y_pred2), 3)
print("Kappa:", kappa, end='\t')
accuracy = round(accuracy_score(y_test, y_pred2), 3)
print("Accuracy:", accuracy, '\n \n')


print('Pipeline 3: DecisionTreeClassifier')
pipe3.fit(X_train, y_train)
y_pred3 = pipe3.predict(X_test)
print(classification_report(y_test, y_pred3, target_names=target_names),'\n \n')
f1 = round(f1_score(y_test, y_pred3), 3)
print("F1 Score:", f1, end='\t')
kappa = round(cohen_kappa_score(y_test, y_pred3), 3)
print("Kappa:", kappa, end='\t')
accuracy = round(accuracy_score(y_test, y_pred3), 3)
print("Accuracy:", accuracy, '\n \n')


Pipeline 1: KNeighborsClassifier
                 precision    recall  f1-score   support

No clasificados       0.95      0.99      0.97      3303
   Clasificados       0.55      0.11      0.19       215

       accuracy                           0.94      3518
      macro avg       0.75      0.55      0.58      3518
   weighted avg       0.92      0.94      0.92      3518
 

F1 Score: 0.185	Kappa: 0.168	Accuracy: 0.94 
 

Pipeline 2: SVC linear
                 precision    recall  f1-score   support

No clasificados       0.96      0.99      0.98      3303
   Clasificados       0.79      0.30      0.44       215

       accuracy                           0.95      3518
      macro avg       0.87      0.65      0.71      3518
   weighted avg       0.95      0.95      0.94      3518
 
 

F1 Score: 0.438	Kappa: 0.418	Accuracy: 0.953 
 

Pipeline 3: DecisionTreeClassifier
                 precision    recall  f1-score   support

No clasificados       0.96      0.96      0.96      3303
 

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?


**Respuesta**:


- Precisión: indica que porcentaje de los jugadores que indicamos como seleccionados nacionales efectivamente son seleccionados nacionales.
- Recall: de la totalidad de jugadores seleccionados nacionales, que porcentaje son capaces los modelos de detectar.
- Accuracy: determina el porcentaje de exactitud de la clasificación para todas las muestras, es decir, cuantas clasificaciones fueron correctas (jugadores clasificados correctamente).

Con respecto a los resultados de los pipelines se puede apreciar que se obtuvo un alto valor de accuracy, sin embargo, el valor de F1-score, el cual representa de forma ponderada recall y precision sobre la clase positiva, es bajo, esto debido a los valores de recall y precision obtenidos. Esto indica que el modelo no es capaz de identificar de buena manera los jugadores que son clasificados nacionales, esto puede deberse posiblemente por el desbalance de clases, siendo la clase positiva la que tiene menor cantidad de datos y la negativa teniendo un f1-score de 0.97, 0.98 y 0.96 respectivamente.

Por lo general la cantidad de ejemplos por clases y el desbalance de las mismas afecta el rendimiento de un clasificador, como se aprecia en los resultados anteriores. Lo anterior, por lo general puede mejorarse tomando buenas prácticas como un balanceo de clases mediante UnderSampling u OverSampling, o incluso aumentar la cantidad de data para la clase minoritaria. 


## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [9]:
df_players_2 = pd.read_csv('stats_players.csv')
df_players_2['Labels'] = df_players_2['Club_Position']

# Dado que Club y national position son subconjuntos de las labels se eliminan.
del df_players_2['Club_Position']
del df_players_2['National_Position']

ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']

df_players_2 = df_players_2.loc[df_players_2['Labels'].isin(ataque+central_ataque+central+central_defensa+defensa+arquero),:]
df_players_2.loc[df_players_2['Labels'].isin(ataque),'Labels']= 'ataque'
df_players_2.loc[df_players_2['Labels'].isin(central_ataque),'Labels']= 'central_ataque'
df_players_2.loc[df_players_2['Labels'].isin(central),'Labels']= 'central'
df_players_2.loc[df_players_2['Labels'].isin(central_defensa),'Labels']= 'central_defensa'
df_players_2.loc[df_players_2['Labels'].isin(defensa),'Labels']= 'defensa'
df_players_2.loc[df_players_2['Labels'].isin(arquero),'Labels']= 'arquero'


La cantidad de valores por cada una de las clases descritas previamente son:

In [10]:
df_players_2['Labels'].value_counts()

defensa            1180
central             907
arquero             632
central_ataque      581
ataque              430
central_defensa     209
Name: Labels, dtype: int64

In [11]:
coltransf2 = ColumnTransformer([
    ("MinMaxScaler",MinMaxScaler(),['Age','Shot_Power','Finishing','Long_Shots','Freekick_Accuracy','Penalties']),
    ("StandardScaler",StandardScaler(),['Height','Weight','Ball_Control','Dribbling','Aggression','Reactions','Vision',
                                        'Composure','Crossing','Short_Pass','Long_Pass','Acceleration','Speed','Stamina',
                                        'Balance','Agility','Jumping','Heading','Volleys']),
    ("OneHot",OneHotEncoder(handle_unknown='ignore'),['Nationality','Preffered_Foot','Work_Rate','Weak_foot','Skill_Moves']),
    ("PowerTransform",PowerTransformer(),['Marking','Sliding_Tackle','Standing_Tackle','Interceptions','Strength','Curve'])
])

In [12]:
pipe1 = Pipeline([("Preprocess",coltransf2),
      ('clf', KNeighborsClassifier())])

pipe2 = Pipeline([("Preprocess",coltransf2),
      ('clf', SVC(kernel='linear',probability=True,random_state=1))])

pipe3 = Pipeline([("Preprocess",coltransf2),
      ('clf', DecisionTreeClassifier(criterion='gini',random_state=1))])

De lo anterior, igual se aprecia un desbalance de clases pero mucho menor al caso anterior. Para la clasificación se utilizan los mismos pipelines anteriores.

In [13]:
X = df_players_2.iloc[:,1:-1]
y = df_players_2['Labels']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y, random_state=1)

target_names = df_players_2['Labels'].unique()

print('Pipeline 1: KNeighborsClassifier')
pipe1.fit(X_train, y_train)
y_pred1 = pipe1.predict(X_test)
print(classification_report(y_test, y_pred1, target_names=target_names),'\n')
f1 = round(f1_score(y_test, y_pred1,average='macro'), 3)
print("F1 Score:", f1, end='\t')
kappa = round(cohen_kappa_score(y_test, y_pred1), 3)
print("Kappa:", kappa, end='\t')
accuracy = round(accuracy_score(y_test, y_pred1), 3)
print("Accuracy:", accuracy, '\n \n')


print('Pipeline 2: SVC linear')
pipe2.fit(X_train, y_train)
y_pred2 = pipe2.predict(X_test)
print(classification_report(y_test, y_pred2, target_names=target_names),'\n \n')
f1 = round(f1_score(y_test, y_pred2,average='macro'), 3)
print("F1 Score:", f1, end='\t')
kappa = round(cohen_kappa_score(y_test, y_pred2), 3)
print("Kappa:", kappa, end='\t')
accuracy = round(accuracy_score(y_test, y_pred2), 3)
print("Accuracy:", accuracy, '\n \n')


print('Pipeline 3: DecisionTreeClassifier')
pipe3.fit(X_train, y_train)
y_pred3 = pipe3.predict(X_test)
print(classification_report(y_test, y_pred3, target_names=target_names),'\n \n')
f1 = round(f1_score(y_test, y_pred3,average='macro'), 3)
print("F1 Score:", f1, end='\t')
kappa = round(cohen_kappa_score(y_test, y_pred3), 3)
print("Kappa:", kappa, end='\t')
accuracy = round(accuracy_score(y_test, y_pred3), 3)
print("Accuracy:", accuracy, '\n \n')


Pipeline 1: KNeighborsClassifier
                 precision    recall  f1-score   support

 central_ataque       1.00      0.99      1.00       126
         ataque       0.76      0.76      0.76        86
        arquero       0.51      0.52      0.52       182
        defensa       0.40      0.32      0.35       116
        central       0.27      0.10      0.14        42
central_defensa       0.77      0.93      0.85       236

       accuracy                           0.69       788
      macro avg       0.62      0.60      0.60       788
   weighted avg       0.67      0.69      0.67       788
 

F1 Score: 0.602	Kappa: 0.607	Accuracy: 0.693 
 

Pipeline 2: SVC linear
                 precision    recall  f1-score   support

 central_ataque       1.00      0.99      1.00       126
         ataque       0.78      0.85      0.81        86
        arquero       0.55      0.63      0.59       182
        defensa       0.49      0.32      0.39       116
        central       0.56      0.

**Respuesta**:

En general, para los tres pipelines utilizados se obtienen resultados similares donde el accuracy de los modelos son 0.693, 0.731 y 0.656 respectivamente, los cuales se reducen con respecto al problema anterior, pero se puede verificar mediante el F1-score que son más representativos, esto dado que el F1-score para cada modelo es de 0.602, 0.663 y 0.597 respectivamente, siendo mucho más altos que para el problema anterior. Esto se pude deber a que hay un menor desbalance de clases respecto al de seleccionados nacionales, sin embargo, de las 6 clases existe todavía un gran desbalance entre la de menor cantidad (209 central_defensa) y la de mayor cantidad (1180 defensa). En general, los resultados obtenidos se pueden deber a las estadísticas que tiene cada posición de juego, llegando en ocasiones a confundirse entre éstas, dado que presentan valores similares probablemente.

Nuevamente, para poder mejorar el rendimiento de los modelos la mejor opción sería aplicar un balanceo de clases como los mencionados anteriormente.

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

In [48]:
df_players_3 = pd.read_csv('stats_players.csv')
df_sueldos = pd.read_csv('salarios.csv',index_col=0)
df_sueldos.rename(columns = {'Player':'Name'}, inplace = True)
df_total = df_players_3.merge(df_sueldos, how='inner', on='Name')

In [59]:
coltransf2 = ColumnTransformer([
    ("MinMaxScaler",MinMaxScaler(),['Age','Shot_Power','Finishing','Long_Shots','Freekick_Accuracy','Penalties']),
    ("StandardScaler",StandardScaler(),['Height','Weight','Ball_Control','Dribbling','Aggression','Reactions','Vision',
                                        'Composure','Crossing','Short_Pass','Long_Pass','Acceleration','Speed',
                                        'Balance','Agility','Jumping','Heading','Volleys']),
    ("OneHot",OneHotEncoder(handle_unknown='ignore'),['Nationality','National_Position','Skill_Moves']),
    ("PowerTransform",PowerTransformer(),['Marking','Sliding_Tackle','Standing_Tackle','Interceptions','Strength','Curve'])
])

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge

# Club_position, weak_foot, preffered foot, Work_Rate, stamina
pipe_regressor = Pipeline([("Preprocess",coltransf2),
      ('regressor',ElasticNet(alpha=0.01))])

In [60]:
X = df_total.iloc[:,1:-1]
y = df_total['Weekly Salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=1)


print('Pipeline: LinearRegression')
pipe_regressor.fit(X_train, y_train)
y_pred_reg = pipe_regressor.predict(X_test)

R2 = r2_score(y_test, y_pred_reg)

print(f'Valor de R2: {R2}')


Pipeline: LinearRegression
Valor de R2: 0.3864944349298396


**Respuesta**:
- R2: es el coeficiente de regresión, que indica la capacidad de un modelo para predecir futuros resultados. El mejor resultado posible es 1.0, y ocurre cuando la predicción coincide con los valores de la variable objetivo. R2 puede tomar valores negativos pues la predicción puede ser arbitrariamente mala. Cuando la predicción coincide con la esperanza de los valores de la variable objetivo, el resultado de R2 es 0. 

Con respecto a los resultados obtenidos se tiene un valor de $R2= 0.3865$, lo que indica que los valores predichos para el conjunto de testeo difieren considerablemente de los valores reales, esto se puede deber en primera instancia, por la baja cantidad de datos que se ocupan para entrenar el modelo de regresión. Además, es posible que existan sueldos con mucha predominancia en cierto rango, por lo que la regresión se ajusta en torno a esos valores. 

Incluir una mayor cantidad de datos probablemente mejoren el rendimiento del modelo. Por otro lado, incluir nuevas características que sean de gran importancia para el problema en cuestión (fútbol) tales como el club y la liga a la que pertenecen, o incluso la cantidad de distinciones individuales, pueden ayudar a representar de mejor forma la importancia de cada jugador y, por ende, su costo de mercado.




# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>